## Model Training And EDA for Netflix Stock

# Importing Depandancies

In [41]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

# EDA

In [42]:
df =pd.read_csv("Netflix_Stock_Price.csv")

In [43]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
1,2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2,2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
3,2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
4,2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900


In [44]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1009 non-null   object 
 1   Open       1009 non-null   float64
 2   High       1009 non-null   float64
 3   Low        1009 non-null   float64
 4   Close      1009 non-null   float64
 5   Adj Close  1009 non-null   float64
 6   Volume     1009 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 55.3+ KB


In [46]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1.009000e+03
mean,419.059673,425.320703,412.374044,419.000733,419.000733,7.570685e+06
std,108.537532,109.262960,107.555867,108.289999,108.289999,5.465535e+06
min,233.919998,250.649994,231.229996,233.880005,233.880005,1.144000e+06
25%,331.489990,336.299988,326.000000,331.619995,331.619995,4.091900e+06
50%,377.769989,383.010010,370.880005,378.670013,378.670013,5.934500e+06
75%,509.130005,515.630005,502.529999,509.079987,509.079987,9.322400e+06
max,692.349976,700.989990,686.090027,691.690002,691.690002,5.890430e+07


In [47]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
1,2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2,2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
3,2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
4,2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900


In [48]:
df.sample(5)

,Date,Open,High,Low,Close,Adj Close,Volume
186,2018-10-30,275.570007,290.519989,271.209991,285.809998,285.809998,23685700
880,2021-08-04,513.000000,517.979980,510.369995,517.349976,517.349976,2039400
91,2018-06-15,390.709991,398.859985,387.510010,391.980011,391.980011,13588100
14,2018-02-26,288.750000,295.649994,287.010010,294.160004,294.160004,10268600
191,2018-11-06,314.760010,320.220001,305.299988,310.839996,310.839996,9710400


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1009 non-null   object 
 1   Open       1009 non-null   float64
 2   High       1009 non-null   float64
 3   Low        1009 non-null   float64
 4   Close      1009 non-null   float64
 5   Adj Close  1009 non-null   float64
 6   Volume     1009 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 55.3+ KB


In [50]:
df = df.drop(columns=['Date','Volume'], axis=1)

In [51]:
df.head()

,Open,High,Low,Close,Adj Close
0,262.000000,267.899994,250.029999,254.259995,254.259995
1,247.699997,266.700012,245.000000,265.720001,265.720001
2,266.579987,272.450012,264.329987,264.559998,264.559998
3,267.079987,267.619995,250.000000,250.100006,250.100006
4,253.850006,255.800003,236.110001,249.470001,249.470001


# Independent and Dependent Features

In [ ]:
X = df.drop(columns=['Close'],axis=1) #Independent
y= df['Close']  #Dependent

# Using Train Test Split

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=10)

In [54]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((676, 4), (676,), (333, 4), (333,))

# Using Column Transform for preprocessing

In [55]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [56]:
X = preprocessor.fit_transform(X)
X = pd.DataFrame(X)

# Model Training and Evaluating

In [57]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [58]:
from sklearn.linear_model import Lasso

# Train using only Lasso Regression
lasso_model = Lasso()
lasso_model.fit(X_train, y_train)

# Make predictions
y_train_pred = lasso_model.predict(X_train)
y_test_pred = lasso_model.predict(X_test)

# Evaluate model performance
train_mae, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
test_mae, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)

print("Lasso Regression Performance:")
print("Training Set:")
print(f"- Root Mean Squared Error: {train_rmse:.4f}")
print(f"- Mean Absolute Error: {train_mae:.4f}")
print(f"- R2 Score: {train_r2:.4f}")

print("\nTest Set:")
print(f"- Root Mean Squared Error: {test_rmse:.4f}")
print(f"- Mean Absolute Error: {test_mae:.4f}")
print(f"- R2 Score: {test_r2:.4f}")

Lasso Regression Performance:
Training Set:
- Root Mean Squared Error: 1.0635
- Mean Absolute Error: 0.7568
- R2 Score: 0.9999

Test Set:
- Root Mean Squared Error: 1.1198
- Mean Absolute Error: 0.7671
- R2 Score: 0.9999


C:\Users\ds075\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.122e+03, tolerance: 7.728e+02
  model = cd_fast.enet_coordinate_descent(


# Exporting the model in pickel file

In [59]:
# Save the Lasso model
import pickle
pickle.dump(lasso_model, open('lasso_model.pkl', 'wb'))